In [5]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [6]:
true_w=torch.tensor([2,-3.4])
true_b=4.2
features,labels=d2l.synthetic_data(true_w,true_b,1000)

In [7]:
def load_array(data_arrays,batch_size,is_trai=True):
    dataset=data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset,batch_size,shuffle=is_train)

In [8]:
batch_size=10
data_iter=load_array((features,labels),batch_size)

TypeError: load_array() missing 1 required positional argument: 'is_train'